# Differentiation

Compute derivatives, first by finite differences, then by automatic differentiation.

These formulas are based on the definition of the derivative:

$$f'(x) = \lim_{h\to 0}\frac{f(x+h)-f(x)}{h}.$$

In [8]:
forward_diff(f, x0, h) = (f(x0 + h) - f(x0) )/h
backward_diff(f, x0, h) = (f(x0) - f(x0-h))/h
centred_diff(f, x0, h) = (f(x0+h)-f(x0-h))/(2*h)

centred_diff (generic function with 1 method)

In [5]:
@show forward_diff(sin, 0, 0.01)
@show backward_diff(sin, 0, 0.01)
@show centred_diff(sin, 0, 0.01);

forward_diff(sin, 0, 0.01) = 0.9999833334166665
backward_diff(sin, 0, 0.01) = 0.9999833334166665
centred_diff(sin, 0, 0.01) = 0.9999833334166665


In [6]:
@show forward_diff(exp, 2, 0.01)
@show backward_diff(exp, 2, 0.01)
@show centred_diff(exp, 2, 0.01);

forward_diff(exp, 2, 0.01) = 7.426124838854253
backward_diff(exp, 2, 0.01) = 7.352233662108354
centred_diff(exp, 2, 0.01) = 7.389179250481304


## Automatic differentiation

Create an ordered pair of a number $x$ and the derivative of a function evaluated at $x$. Use algebraic differentiation rules to define elementary functions on these ordered pairs ("dual numbers").

See:

* https://www.juliabloggers.com/automatic-differentiation-with-dual-numbers/
* [Video](https://www.youtube.com/watch?v=vAp6nUMrKYg)
* [ForwardDiff documentation](https://juliadiff.org/ForwardDiff.jl/v0.8/dev/how_it_works.html)

In [13]:
struct DN   # "dual number"
    val
    deriv
end

Redefine plus, minus, product, quotient, exponentiation, etc.

In [14]:
Base.:+(a::DN, b::DN) = DN(a.val + b.val, a.deriv .+ b.deriv)
Base.:*(a::DN, b::DN) = DN(a.val * b.val, b.val .* a.deriv .+ a.val .* b.deriv)
Base.:^(a::DN, b) = DN(a.val^b, b .* a.val .^ (b .- 1) .* a.deriv);

Compute the derivative of a function of a, b, and c with respect to a. Make the derivative of a = 1 and the derivatives of b and c 0.

In [21]:
a = DN(1.0, 1.0)
b = DN(0.5, 0.0)
c = DN(2.0, 0.0);

Evaluate $f(a, b, c) = c(a+2b)^2$.

In [24]:
d = c*(a + DN(2.0, 0.0)*b)^2  # c(a+b)*2*1 

DN(8.0, 8.0)

Now get derivatives with respect to each variable.

In [26]:
a = DN(1.0, [1.0, 0.0, 0.0])
b = DN(0.5, [0.0, 1.0, 0.0])
c = DN(2.0, [0.0, 0.0, 1.0])
d = c*(a + DN(2.0, 0.0)*b)^2  

DN(8.0, [8.0, 16.0, 4.0])

Check. Compute d(a,b,c) and d/da, d/db, d/dc.

In [29]:
df(a,b,c) = c*(a+2*b)^2
dda(a,b,c) = c*(a+2*b)*2
ddb(a,b,c) = c*(a+2*b)*2*2
ddc(a,b,c) = (a+2*b)^2
@show df(1, 0.5, 2)
@show dda(1, 0.5, 2)
@show ddb(1, 0.5, 2)
@show ddc(1, 0.5, 2);

df(1, 0.5, 2) = 8.0
dda(1, 0.5, 2) = 8.0
ddb(1, 0.5, 2) = 16.0
ddc(1, 0.5, 2) = 4.0


Define sin, cos, exp, log, abs, etc. in the same way.

Until that's done, can't compute sin(a), a^a, log(a), exp(a), etc.

For a complete implementation of this idea, use the ForwardDiff library.

In [2]:
using ForwardDiff

In [20]:
ForwardDiff.derivative(a -> 2.0*(a+0.5)^2, 1.0)

6.0

In [3]:
f(x) = sin(exp(x) + 4*log(x))
ForwardDiff.derivative(f, pi)

-20.752991086118303

Check: $\frac{d}{dx} \sin(e^x+4\log x) = \cos(e^x + 4\log x)*(e^x+4/x)$.

In [6]:
df(x) = cos(exp(x)+4*log(x))*(exp(x)+4/x)
@show df(pi)
@show df(pi) - ForwardDiff.derivative(f, pi);

df(pi) = -20.752991086118303
df(pi) - ForwardDiff.derivative(f, pi) = 0.0


In [12]:
@show centred_diff(f, pi, 1e-5)
@show centred_diff(f, pi, 1e-5) - ForwardDiff.derivative(f, pi);

centred_diff(f, pi, 1.0e-5) = -20.75299089493443
centred_diff(f, pi, 1.0e-5) - ForwardDiff.derivative(f, pi) = 1.9118387228900247e-7


Which method requires more computations? Which takes longer?

In [26]:
@time for i=1:100000
        centred_diff(f, pi*i/1000, 0.00001)
end

  0.012523 seconds


In [27]:
@time for i=1:100000
        ForwardDiff.derivative(f, pi*i/1000)
end

  0.006777 seconds


More exotic examples and some unexpected results.

In [31]:
g(x) = x^x
@show ForwardDiff.derivative(g, 2)  # log y = x log x; y' = y (1 + log x)
@show 2^2 * (1 + log(2));

ForwardDiff.derivative(g, 2) = 6.772588722239782
2 ^ 2 * (1 + log(2)) = 6.772588722239782


# 

In [46]:
@show ForwardDiff.derivative(log, 0)
@show ForwardDiff.derivative(abs, 0)
h1(x) = 1/x
@show ForwardDiff.derivative(h1, 0)
h2(x) = sin(x)/x
@show ForwardDiff.derivative(h2, 0);

ForwardDiff.derivative(log, 0) = Inf
ForwardDiff.derivative(abs, 0) = 1
ForwardDiff.derivative(h1, 0) = -Inf
ForwardDiff.derivative(h2, 0) = NaN


## Higher order derivatives

To get a second or higher order derivative, compose the derivative function with itself.

In [9]:
f(x) = 3*x^4 - x^2 + 1
df(x) = ForwardDiff.derivative(f,x)
d2f(x) = ForwardDiff.derivative(df,x)
d3f(x) = ForwardDiff.derivative(d2f,x)
d4f(x) = ForwardDiff.derivative(d3f,x)
@show f(2)
@show df(2)
@show d2f(2)
@show d3f(2)
@show d4f(2)

f(2) = 45
df(2) = 92
d2f(2) = 142
d3f(2) = 144
d4f(2) = 72


72